In [ ]:
# 1. FINAL_CODE
from common_imports import *
show_home_button()
from db_connection import get_engine

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
class DatabaseManager:
    def __init__(self):
        try:
            self.engine = get_engine(autocommit=True)
            logging.info("SQL-engine succesvol aangemaakt.")
        except Exception as e:
            logging.exception("Fout bij engine-creatie")
            raise

    def execute_update(self, sql_text: str, parameters: dict | None = None):
        try:
            with self.engine.connect() as conn:
                conn.execute(text(sql_text), parameters or {})
                conn.commit()
        except SQLAlchemyError as e:
            logging.error("Fout tijdens update: %s  | Params=%s", e, parameters)
            raise

    def fetch_one(self, sql_text: str, parameters: dict | None = None):
        try:
            with self.engine.connect() as conn:
                return conn.execute(text(sql_text), parameters or {}).fetchone()
        except SQLAlchemyError as e:
            logging.error("Fout tijdens fetch_one: %s  | Params=%s", e, parameters)
            raise

    def fetch_dataframe(
        self,
        sql_text: str,
        parameters: dict | None = None,
        parse_dates: list[str] | None = None
    ):
        try:
            with self.engine.connect() as conn:
                return read_sql_query(
                    text(sql_text), conn,
                    params=parameters or {},
                    parse_dates=parse_dates
                )
        except SQLAlchemyError as e:
            logging.error("Fout tijdens fetch_dataframe: %s  | Params=%s", e, parameters)
            raise
class BusinessLogic:
    def __init__(self, db_manager: DatabaseManager):
        self.db = db_manager

    def calculate_factor(self, uprog, iprog, uprim, usec, iprim, isec, netloss, multiplier):
        query = """
        SELECT dbo.CalculateFactorWithoutRegister(
            :uprog, :iprog,
            :uprim, :usec,
            :iprim, :isec,
            :netloss, :multiplier
        ) AS FactorCalc
        """
        try:
            row = self.db.fetch_one(query, {
                "uprog": uprog, "iprog": iprog,
                "uprim": uprim, "usec": usec,
                "iprim": iprim, "isec": isec,
                "netloss": netloss, "multiplier": multiplier
            })
            return row[0] if row else None
        except Exception as e:
            logging.error("Fout bij berekening van factor: %s", e)
            return None
def bulk_update_factors(df: pd.DataFrame, db_manager: DatabaseManager):
    if df.empty:
        logging.info("Geen data in DataFrame; niets te verwerken.")
        return

    insert_sql = sqlalchemy.text("""
        INSERT INTO TBL_Register_Factor_Update (RegisterId, Factor_Nieuw, VanDatum, TotDatum)
        VALUES (:register_id, :factor, CONVERT(SMALLDATETIME, :vandatum), CONVERT(SMALLDATETIME, :totdatum))
    """)
    update_register_sql = sqlalchemy.text("""
        UPDATE TBL_Register
        SET Uprim = :uprim, Usec = :usec, Uprog = :uprog, Iprim = :iprim,
            Isec = :isec, Iprog = :iprog, Netloss = :netloss, Multiplier = :multiplier
        WHERE ID = :register_id
    """)
    records = df.to_dict('records')

    try:
        with db_manager.engine.begin() as conn:
            conn.execute(insert_sql, records)
            logging.info("Bulk insert in TBL_Register_Factor_Update succesvol.")
            conn.execute(sqlalchemy.text("SET NOCOUNT ON; EXEC SP_EDS2_Register_Factor_Update;"))
            logging.info("Stored procedure SP_EDS2_Register_Factor_Update succesvol aangeroepen.")

            reg_ids = df["register_id"].tolist()
            check_sql = sqlalchemy.text("""
                SELECT RegisterId, Omzetdatum FROM TBL_Register_Factor_Update
                WHERE RegisterId IN :reg_ids AND Omzetdatum IS NULL
            """).bindparams(sqlalchemy.bindparam("reg_ids", expanding=True))
            missing = conn.execute(check_sql, {"reg_ids": reg_ids}).fetchall()
            if missing:
                for row in missing:
                    logging.error(f"RegisterID {row['RegisterId']} heeft geen Omzetdatum.")
                raise Exception("Factorupdate incomplete: sommige registers missen Omzetdatum.")
            logging.info("Alle registers hebben een Omzetdatum; update OK.")
            conn.execute(update_register_sql, records)
            logging.info("Registerconfiguratie succesvol bijgewerkt voor alle records.")
    except Exception as e:
        logging.error("Bulk update van factoren mislukt: %s", e)
        raise
class UIManager:
    MIN_SMALLDT = datetime(1900, 1, 1)
    MAX_SMALLDT = datetime(2079, 6, 6, 23, 59)
    INTERVAL_MAP = {
        "Week": "W",
        "Day": "D",
        "Hour": "H",
        "Quarter (15min)": "15T",
        "5 minutes": "5T"
    }
    COL_W_LBL = "130px"
    COL_W_HDR = "170px"
    COL_W_WDG = "150px"
    COL_W_REM = "50px"

    def __init__(self, db_manager: DatabaseManager, business_logic: BusinessLogic):
        self.db = db_manager
        self.logic = business_logic
        self.register_widgets = []
        self.setup_widgets()
        self.build_ui()

    def setup_widgets(self):
        self.progress_bar = widgets.IntProgress(value=0, min=0, max=100, step=1,
                                        description='Voortgang:', bar_style='info', continuous_update=False)
        self.status_label = widgets.Label(value="", layout=widgets.Layout(width="auto", margin="0 0 0 10px"))
        self.progress_container = widgets.HBox(
            [self.progress_bar, self.status_label],
            layout=widgets.Layout(visibility='hidden',
                                  align_items='center',
                                  justify_content='center')
        )
        self.output_widget = widgets.Output()
        self.registrator_id_widget = widgets.Text(
            description='RegistratorId:',
            placeholder='Vul RegistratorId in',
            value='',
            layout=widgets.Layout(width='220px')
        )
        self.fetch_reg_button = widgets.Button(
            description='Zoek Registers', button_style='info', icon='database'
        )
        self.register_ids_html = widgets.HTML("<b>Register IDs (found):</b> None yet")

        self.interval_widget = widgets.Dropdown(
            options=list(self.INTERVAL_MAP.keys()),
            value="Hour", description="Interval:"
        )

        self.compare_button = widgets.Button(
            description='Visualiseer', button_style='primary', icon='area-chart'
        )
        self.compare_reading_button = widgets.Button(
            description='Vergelijk', button_style='info', icon='search'
        )
        self.bereken_alle_factoren_btn = widgets.Button(
            description='Bereken Factoren', button_style='info', icon='calculator'
        )

        self.bulk_update_button = widgets.Button(
            description='Factor updaten', button_style='success', icon='rocket'
        )

        self.register_rows_container = widgets.VBox([])
        self.top_box = widgets.HBox(
            [self.registrator_id_widget, self.fetch_reg_button, self.register_ids_html],
            layout=widgets.Layout(gap="10px", align_items='center', flex_flow="row wrap")
        )
        self.button_row = widgets.HBox(
            [
                self.bereken_alle_factoren_btn,
                self.bulk_update_button,
                self.interval_widget,
                self.compare_button,
                self.compare_reading_button
            ],
            layout=widgets.Layout(gap="10px", align_items='center', flex_flow="row wrap")
        )
        self.filters_container = widgets.VBox(
            [self.top_box, self.register_rows_container, self.button_row,
             self.progress_container, self.output_widget],
            layout=widgets.Layout(width='100%', padding="10px")
        )
        self.toggle_filters_button = widgets.Button(
            description="Verberg", icon='chevron-up', button_style='info'
        )
        self.toggle_filters_button.on_click(self.toggle_filters_display)

        self.view_container = widgets.VBox([], layout=widgets.Layout(width='100%'))
        self.main_ui = widgets.VBox(
            [self.toggle_filters_button, self.filters_container, self.view_container],
            layout=widgets.Layout(width='100%', height='auto', padding="10px")
        )

        self.fetch_reg_button.on_click(self.on_fetch_reg_button_clicked)
        self.bereken_alle_factoren_btn.on_click(self.on_bereken_alle_factoren_clicked)
        self.bulk_update_button.on_click(self.on_bulk_update_button_clicked)
        self.compare_button.on_click(self.on_compare_button_clicked)
        self.compare_reading_button.on_click(self.on_compare_reading_button_clicked)
    def build_ui(self):
        display(self.main_ui)
    def show_progress(self, msg="", initial_value=0):
        self.progress_bar.bar_style = 'info'
        self.progress_container.layout.visibility = 'visible'
        self.progress_bar.value = initial_value
        self.status_label.value = msg

    def update_progress(self, value, msg="", error=False):
        self.progress_bar.value = value
        self.status_label.value = msg
        if error:
            self.progress_bar.bar_style = "danger"
        elif value >= 100:
            self.progress_bar.bar_style = "success"

    def finish_progress(self):
        time.sleep(1)
        self.progress_container.layout.visibility = 'hidden'
        self.progress_bar.value = 0
        self.status_label.value = ""

    def toggle_filters_display(self, _):
        disp = self.filters_container.layout.display
        if disp == 'none':
            self.filters_container.layout.display = 'flex'
            self.toggle_filters_button.description = "Verberg"
            self.toggle_filters_button.icon = "chevron-up"
        else:
            self.filters_container.layout.display = 'none'
            self.toggle_filters_button.description = "Toon filters"
            self.toggle_filters_button.icon = "chevron-down"

    def validate_vandatum(self, sender):
        try:
            dt_val = pd.to_datetime(sender.value)
            if dt_val < self.MIN_SMALLDT or dt_val > self.MAX_SMALLDT:
                raise ValueError("Datum buiten SMALLDATETIME bereik.")
            sender.value = dt_val.strftime("%Y-%m-%d %H:%M")
        except Exception as e:
            with self.output_widget:
                print(f"[FOUT] Datuminvoer: {e}")

    def build_register_table(self):
        with self.output_widget:
            clear_output(wait=True)
        if not self.register_widgets:
            self.register_rows_container.children = []
            self.register_ids_html.value = "<b>Register IDs (found):</b> (Geen registers)"
            return

        rlist = [r['register_id'] for r in self.register_widgets]
        self.register_ids_html.value = f"<b>Register IDs (found):</b> {rlist}"

        fields = [
            ("Current Factor", "current_factor_label"),
            ("Uprim", "uprim"),
            ("Usec", "usec"),
            ("Uprog", "uprog"),
            ("Iprim", "iprim"),
            ("Isec", "isec"),
            ("Iprog", "iprog"),
            ("Netloss", "netloss_invoer"),
            ("Multiplier", "multiplier_invoer"),
            ("VanDatum", "vdatum"),
            ("TotDatum", "toddatum"),
            ("TestMode", "testmode"),
            ("New Factor", "factor_label"),
            ("Remove", "remove_button"),
        ]

        hdrs = [widgets.HTML("<b>Velden</b>", layout=widgets.Layout(width=self.COL_W_LBL))]
        num_registers = len(self.register_widgets)

        available_width_for_registers_px = 0
        try:
            parent_width_assumption = 900 
            label_col_width_px = float(self.COL_W_LBL.replace("px", ""))
            available_width_for_registers_px = parent_width_assumption * 0.98 - label_col_width_px
        except ValueError:
            pass 

        if num_registers > 0 and available_width_for_registers_px > 0:
            width_per_reg_px = (available_width_for_registers_px / num_registers) - 5 
            calculated_width_px = f"{max(50, width_per_reg_px):.0f}px"
            col_w_hdr_dynamic = calculated_width_px
            col_w_wdg_dynamic = calculated_width_px
        else:
            available_width_percentage = 100 - (float(self.COL_W_LBL.replace("px", "")) / 10 if "px" in self.COL_W_LBL else 15)
            if num_registers > 0 :
                perc_width = f"{available_width_percentage / num_registers * 0.98}%"
                col_w_hdr_dynamic = perc_width
                col_w_wdg_dynamic = perc_width
            else:
                col_w_hdr_dynamic = self.COL_W_HDR
                col_w_wdg_dynamic = self.COL_W_WDG
        
        for reg in self.register_widgets:
            hdrs.append(widgets.HTML(f"<b>{reg['description']}</b><br>(ID={reg['register_id']})",
                                     layout=widgets.Layout(width=col_w_hdr_dynamic)))
        header_row = widgets.HBox(hdrs, layout=widgets.Layout(width='100%', overflow_x='auto',
                                                             align_items='center',
                                                             justify_content='flex-start'))
        rows = []
        for label, key in fields:
            cells = [widgets.Label(label, layout=widgets.Layout(width=self.COL_W_LBL))]
            for reg in self.register_widgets:
                w = reg[key]
                current_cell_width = col_w_wdg_dynamic
                container_layout = widgets.Layout(
                    width=current_cell_width,
                    display='flex',
                    align_items='center',
                    overflow_x='hidden'
                )

                if key == "remove_button":
                    container_layout.justify_content = 'center'
                elif isinstance(w, (widgets.Checkbox)):
                    container_layout.justify_content = 'flex-start'
                    w.layout.margin = '0 0 0 5px' 
                else:
                    container_layout.justify_content = 'flex-start'

                if key == "remove_button":
                    pass
                elif hasattr(w, 'layout') and not isinstance(w, widgets.Button):
                    w.layout.max_width = '98%'
                    w.layout.width = 'auto'
                
                container = widgets.Box([w], layout=container_layout)
                cells.append(container)
            row = widgets.HBox(cells, layout=widgets.Layout(width='100%', overflow_x='auto',
                                                            align_items='center',
                                                            justify_content='flex-start'))
            rows.append(row)
        self.register_rows_container.children = [header_row] + rows
        self.register_rows_container.layout.width = '100%'
    def on_fetch_reg_button_clicked(self, _):
        with self.output_widget:
            clear_output()
        self.register_widgets.clear()
        self.register_rows_container.children = []
        
        registrator_id_str = self.registrator_id_widget.value
        try:
            rid = int(registrator_id_str)
            if rid <= 0:
                raise ValueError("RegistratorId moet een positief getal zijn.")
        except ValueError as e:
            with self.output_widget:
                print(f"Ongeldige RegistratorId: {e if registrator_id_str else 'veld is leeg.'}")
            self.build_register_table() # To clear table if invalid
            return

        self.show_progress("Ophalen register-IDs...", initial_value=0)
        sql = """
        SELECT ID AS RegisterId
        FROM TBL_Register
        WHERE RegistratorId = :rid
        ORDER BY ID
        """
        try:
            df = self.db.fetch_dataframe(sql, {"rid": rid})
        except Exception as e:
            with self.output_widget:
                print("Fout bij ophalen registers:", e)
            self.finish_progress()
            self.build_register_table()
            return

        if df.empty:
            with self.output_widget:
                print("Geen registers voor deze RegistratorId.")
            self.finish_progress()
            self.build_register_table()
            return

        ids = df["RegisterId"].tolist()
        with self.output_widget:
            print("Registers found:", ids)
        for i, reg_id in enumerate(ids):
            self.update_progress(int((i / len(ids)) * 100) if len(ids) > 0 else 0 , f"Laden register {reg_id} ({i+1}/{len(ids)})...")
            try:
                row_desc = self.db.fetch_one(
                    "SELECT Description FROM TBL_Register WHERE ID = :rid", {"rid": reg_id}
                )
                desc = row_desc[0] if row_desc else f"Register {reg_id}"
            except Exception as e_desc:
                logging.warning(f"Kon beschrijving niet ophalen voor register {reg_id}: {e_desc}")
                desc = f"Register {reg_id}"

            ctype = "LDN" if "LDN" in desc.upper() else ("ODN" if "ODN" in desc.upper() else "OTHER")

            try:
                row_factor = self.db.fetch_one(
                    "SELECT Factor FROM TBL_Register WHERE ID = :rid", {"rid": reg_id}
                )
                cf = row_factor[0] if row_factor else None
            except Exception as e_factor:
                logging.warning(f"Kon factor niet ophalen voor register {reg_id}: {e_factor}")
                cf = None
            cf_str = f"{cf:.4f}" if cf is not None else "Onbekend"
            w_current_factor = widgets.Label(cf_str, layout=widgets.Layout(width='auto'))

            try:
                defaults = self.db.fetch_one(
                    """
                    SELECT UPrim, Usec, Uprog, Iprim, Iprog, Isec, Netloss, Multiplier
                    FROM TBL_Register WHERE ID = :rid
                    """, {"rid": reg_id}
                )
            except Exception as e_defaults:
                logging.warning(f"Kon defaults niet ophalen voor register {reg_id}: {e_defaults}")
                defaults = None

            if defaults:
                uprim, usec, uprog_d, iprim, iprog_d, isec, netloss_d, mult_d = defaults
                uprog_d = bool(uprog_d); iprog_d = bool(iprog_d)
            else:
                uprim, usec, uprog_d, iprim, iprog_d, isec, netloss_d, mult_d = 800, 100, True, 2000, False, 5, 1.0, 1.0
            widget_layout = widgets.Layout(width='auto', max_width='98%')
            w_uprim = widgets.IntText(value=uprim, layout=widget_layout, continuous_update=False)
            w_usec  = widgets.IntText(value=usec, layout=widget_layout, continuous_update=False)
            w_uprog = widgets.Checkbox(value=uprog_d, description="", indent=False, layout=widgets.Layout(width='auto'))
            w_iprog = widgets.Checkbox(value=iprog_d, description="", indent=False, layout=widgets.Layout(width='auto'))
            w_iprim = widgets.IntText(value=iprim, layout=widget_layout, continuous_update=False)
            w_isec  = widgets.IntText(value=isec, layout=widget_layout, continuous_update=False)
            w_netloss = widgets.FloatText(value=netloss_d, layout=widget_layout, continuous_update=False)
            w_multiplier = widgets.FloatText(value=mult_d, layout=widget_layout, continuous_update=False)
            now_val = datetime.now()
            now_str = now_val.strftime("%Y-%m-%d %H:%M")
            future_str = (now_val + timedelta(days=370)).strftime("%Y-%m-%d %H:%M")
            w_vdatum = widgets.Text(value=now_str, layout=widget_layout, continuous_update=False)
            w_vdatum.on_submit(self.validate_vandatum)
            w_todatum = widgets.Text(value=future_str, disabled=True, layout=widget_layout, continuous_update=False)

            w_testmode = widgets.RadioButtons(
                options=[('Rollback=0', 0), ('Commit=1', 1)],
                value=1, layout=widgets.Layout(width='auto') # Defaulting to Commit = 1 based on original code
            )
            w_factor_label = widgets.Label("(nog niet berekend)", layout=widgets.Layout(width='auto'))
            w_remove = widgets.Button(description="x", layout=widgets.Layout(width='auto', min_width='30px', height="25px"))
            w_remove.style.button_color = "#D3D3D3"

            rowdict = {
                'register_id': reg_id,
                'description': desc,
                'channel_type': ctype,
                'current_factor_label': w_current_factor,
                'uprim': w_uprim,
                'usec': w_usec,
                'uprog': w_uprog,
                'iprim': w_iprim,
                'isec': w_isec,
                'iprog': w_iprog,
                'netloss_invoer': w_netloss,
                'multiplier_invoer': w_multiplier,
                'vdatum': w_vdatum,
                'toddatum': w_todatum,
                'testmode': w_testmode,
                'factor_label': w_factor_label,
                'remove_button': w_remove
            }

            def make_remove_callback(rd):
                def _cb(_b):
                    with self.output_widget:
                        print(f"Removing register {rd['register_id']}")
                    self.register_widgets.remove(rd)
                    self.build_register_table()
                return _cb

            w_remove.on_click(make_remove_callback(rowdict))
            self.register_widgets.append(rowdict)

        self.build_register_table()
        with self.output_widget:
            print("UI aangemaakt. Klaar voor factorberekening/updating.")
        self.finish_progress()

    def on_bereken_alle_factoren_clicked(self, _):
        with self.output_widget:
            clear_output()
            if not self.register_widgets:
                print("Geen registers in UI.")
                return
        self.show_progress("Factoren berekenen...", initial_value=0)

        if not self.register_widgets: return # Should be caught above but good for safety

        tpl = self.register_widgets[0]
        vals = {
            'uprim': tpl['uprim'].value,
            'usec': tpl['usec'].value,
            'uprog': tpl['uprog'].value,
            'iprim': tpl['iprim'].value,
            'isec': tpl['isec'].value,
            'iprog': tpl['iprog'].value,
            'mult': tpl['multiplier_invoer'].value,
            'vdt': tpl['vdatum'].value
        }
        for rd in self.register_widgets[1:]:
            rd['uprim'].value = vals['uprim']
            rd['usec'].value = vals['usec']
            rd['uprog'].value = vals['uprog']
            rd['iprim'].value = vals['iprim']
            rd['isec'].value = vals['isec']
            rd['iprog'].value = vals['iprog']
            rd['multiplier_invoer'].value = vals['mult']
            rd['vdatum'].value = vals['vdt']

        num_registers = len(self.register_widgets)
        for i, rd in enumerate(self.register_widgets):
            net = rd['netloss_invoer'].value
            mul = rd['multiplier_invoer'].value
            up = 1 if rd['uprog'].value else 0
            ip = 1 if rd['iprog'].value else 0
            try:
                factor = self.logic.calculate_factor(
                    uprog=up, iprog=ip,
                    uprim=rd['uprim'].value, usec=rd['usec'].value,
                    iprim=rd['iprim'].value, isec=rd['isec'].value,
                    netloss=net, multiplier=mul
                )
                rd['factor_label'].value = f"{factor:.4f}" if factor is not None else "Fout"
            except Exception as e_calc:
                logging.error(f"Fout bij berekenen factor voor register {rd['register_id']}: {e_calc}")
                rd['factor_label'].value = "Fout"
            self.update_progress(int((i + 1) * (100 / num_registers)) if num_registers > 0 else 100, msg=f"Factor {i+1}/{num_registers} berekend...")

        with self.output_widget:
            print("Factoren opnieuw berekend.")
        self.finish_progress()

    def on_commit_bulk_button_clicked(self, _):
        if not self.register_widgets:
            with self.output_widget:
                clear_output()
                print("Geen registers in UI.")
            return
        records = []
        for rd in self.register_widgets:
            ft = rd['factor_label'].value
            fv = None
            if ft != "Fout" and ft != "(nog niet berekend)":
                try:
                    fv = float(ft)
                except ValueError:
                    with self.output_widget:
                        print(f"Ongeldige factor '{ft}' voor register {rd['register_id']}. Update afgebroken.")
                    return
            elif ft == "Fout":
                with self.output_widget:
                    print(f"Factorberekening mislukt voor register {rd['register_id']}. Update afgebroken.")
                return
            else: # (nog niet berekend)
                with self.output_widget:
                    print(f"Factor nog niet berekend voor register {rd['register_id']}. Bereken eerst factoren. Update afgebroken.")
                return

            records.append({
                "register_id": rd['register_id'],
                "factor": fv,
                "vandatum": rd['vdatum'].value,
                "totdatum": rd['toddatum'].value,
                "uprim": rd['uprim'].value,
                "usec": rd['usec'].value,
                "uprog": 1 if rd['uprog'].value else 0,
                "iprim": rd['iprim'].value,
                "isec": rd['isec'].value,
                "iprog": 1 if rd['iprog'].value else 0,
                "netloss": rd['netloss_invoer'].value,
                "multiplier": rd['multiplier_invoer'].value
            })
        df_commit = pd.DataFrame(records)
        with self.output_widget:
            clear_output()
            print("Start DEFINITIEVE bulk update via SP...")
        self.show_progress("Bulk update COMMIT...", initial_value=0)
        try:
            bulk_update_factors(df_commit, self.db)
            self.update_progress(100, "Bulk update succesvol.")
            with self.output_widget:
                print("Bulk update via SP en config-update succesvol afgerond.")
        except Exception as e:
            self.update_progress(self.progress_bar.value, f"Fout: {e}", error=True)
            with self.output_widget:
                print("Bulk update mislukt:", e)
        finally:
            self.finish_progress()

    def on_bulk_update_button_clicked(self, _):
        if not self.register_widgets:
            with self.output_widget:
                clear_output()
                print("Geen registers in UI.")
            return
        self.on_commit_bulk_button_clicked(_)

    def on_compare_button_clicked(self, b):
        with self.output_widget:
            clear_output()
            if not self.register_widgets:
                print("Geen registers in UI.")
                return
        self.view_container.children = []
        if not self.register_widgets:
            return

        row0 = self.register_widgets[0]
        reg_id = row0['register_id']

        try:
            new_factor_val = float(row0['factor_label'].value.strip())
        except ValueError:
            with self.output_widget:
                print(f"Fout: geen geldige berekende factor voor RegisterID={reg_id}. Bereken eerst de factor.")
            return

        vdt_str = row0['vdatum'].value
        try:
            eff_date = pd.to_datetime(vdt_str)
        except ValueError:
            with self.output_widget:
                print(f"Fout: ongeldige VanDatum '{vdt_str}' voor RegisterID={reg_id}.")
            return

        query = """
        SELECT d.period, d.consumption, r.Factor AS current_factor
        FROM TBL_Data d
        JOIN TBL_Register r ON r.ID = d.RegisterID
        WHERE r.ID = :rid
        ORDER BY d.period
        """
        self.show_progress(f"Data ophalen voor plot (RegisterID {reg_id})...", 0)
        try:
            df = self.db.fetch_dataframe(query, {"rid": reg_id}, parse_dates=["period"])
            self.update_progress(30, f"Data opgehaald, verwerken...")
        except Exception as e:
            with self.output_widget:
                print("Fout bij ophalen van data:", e)
            self.finish_progress()
            return

        if df.empty:
            with self.output_widget:
                print(f"Geen data gevonden voor RegisterID={reg_id}.")
            self.finish_progress()
            return

        df["original_consumption_kwh"] = df["consumption"] * df["current_factor"]
        df["adjusted_consumption_kwh"] = np.where(
            df["period"] >= eff_date,
            df["consumption"] * new_factor_val,
            df["original_consumption_kwh"]
        )
        self.update_progress(60, "Data verwerkt, grafiek genereren...")
        plot_start_date = eff_date - pd.Timedelta(days=7)
        plot_end_date = eff_date + pd.Timedelta(days=7)
        max_data_date = df["period"].max()
        today_date_naive = datetime.now().replace(tzinfo=None)
        today_timestamp_naive = pd.Timestamp(today_date_naive.date())
        plot_end_date = min(plot_end_date, max_data_date, today_timestamp_naive + pd.Timedelta(days=1))
        df_plot = df[(df["period"] >= plot_start_date) & (df["period"] <= plot_end_date)].copy()

        if df_plot.empty:
            with self.output_widget:
                print(f"Geen data in het geselecteerde plotvenster ({plot_start_date.date()} tot {plot_end_date.date()}) voor RegisterID={reg_id}.")
            self.finish_progress()
            return

        freq_code = self.INTERVAL_MAP.get(self.interval_widget.value, "H")

        df_plot = df_plot.set_index("period")
        resampled = df_plot[["original_consumption_kwh","adjusted_consumption_kwh"]]\
            .resample(freq_code).sum().dropna(how="all")

        if resampled.empty:
            with self.output_widget:
                print(f"Geen data na resampling met interval '{self.interval_widget.value}' voor RegisterID={reg_id}.")
            self.finish_progress()
            return

        fig_time = go.FigureWidget(
            layout=go.Layout(
                autosize=True,
                title=dict(text=f"Vergelijkingsplot Factor: RegisterID={reg_id}<br>Nieuwe Factor: {new_factor_val:.4f} vanaf {eff_date.strftime('%Y-%m-%d %H:%M')}", x=0.5, xanchor='center'),
                xaxis=dict(title="Tijd", tickangle=-45),
                yaxis=dict(title="Energie (kWh)"),
                template="plotly_white",
                hovermode="x unified",
                hoverlabel=dict(bgcolor='rgba(0,0,0,0.8)', font=dict(color='white')),
                legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1,
                            bgcolor='rgba(255,255,255,0.7)', bordercolor='Black', borderwidth=1),
                height=600,
                margin=dict(l=50, r=20, t=80, b=80),
                dragmode='zoom'
            )
        )

        fig_time.add_trace(
            go.Scatter(
                x=resampled.index, y=resampled["original_consumption_kwh"],
                mode='lines+markers', name='Origineel Verbruik (kWh)'
            )
        )
        fig_time.add_trace(
            go.Scatter(
                x=resampled.index, y=resampled["adjusted_consumption_kwh"],
                mode='lines+markers', name=f'Aangepast Verbruik (kWh)',
                line=dict(dash='dash')
            )
        )
        
        fig_time.add_shape(type="line",
                           x0=eff_date, y0=0,
                           x1=eff_date, y1=1,
                           yref="paper", # Spans the full height relative to y-axis
                           line=dict(color="green", width=2, dash="dash"))
        
        fig_time.add_annotation(x=eff_date, 
                                y=1.0, # Annotation at the top of the plot
                                yref="paper", # y-coordinate is relative to plotting area
                                text="Ingangsdatum nieuwe factor",
                                showarrow=False,
                                font=dict(color="green"), # Match line color
                                align="left",
                                xanchor="left", # Text starts to the right of 'x' coordinate
                                yanchor="top",  # Text's top is at 'y' coordinate
                                xshift=5,       # Shift 5px right from the line
                                yshift=-5       # Shift 5px down from the top edge
                               )

        self.view_container.children = [fig_time]
        self.update_progress(100, "Grafiek gegenereerd.")
        self.finish_progress()

    def on_compare_reading_button_clicked(self, b):
        with self.output_widget:
            clear_output()
            if not self.register_widgets:
                print("Geen registers in UI.")
                return
        self.view_container.children = []
        if not self.register_widgets: return

        row0 = self.register_widgets[0]
        reg_id = row0['register_id']

        try:
            new_factor_val = float(row0['factor_label'].value.strip())
        except ValueError:
            with self.output_widget:
                print(f"Fout: geen geldige berekende factor voor RegisterID={reg_id}. Bereken eerst de factor.")
            return

        vdt_str = row0['vdatum'].value
        try:
            eff_date = pd.to_datetime(vdt_str)
        except ValueError:
            with self.output_widget:
                print(f"Fout: ongeldige VanDatum '{vdt_str}' voor RegisterID={reg_id}.")
            return

        query = """
        SELECT period, consumption, NettLoss, Multiplier, Factor AS current_factor
        FROM TBL_Data d
        JOIN TBL_Register r ON r.ID = d.RegisterID
        WHERE registerid = :reg_id
        ORDER BY period
        """
        self.show_progress(f"Data ophalen voor standenvergelijk (RegisterID {reg_id})...", 0)
        try:
            df = self.db.fetch_dataframe(query, {"reg_id": reg_id}, parse_dates=["period"])
            self.update_progress(30, "Data opgehaald, verwerken...")
        except Exception as e:
            with self.output_widget:
                print("Fout bij ophalen van data:", e)
            self.finish_progress()
            return

        if df.empty:
            with self.output_widget:
                print(f"Geen data gevonden voor RegisterID={reg_id}.")
            self.finish_progress()
            return

        df_display = df[df["period"] >= eff_date].copy()
        if df_display.empty:
            with self.output_widget:
                print(f"Geen data gevonden vanaf ingangsdatum {eff_date.strftime('%Y-%m-%d %H:%M')} voor RegisterID={reg_id}.")
            self.finish_progress()
            return

        df_display["Orig_Stand_kWh"] = df_display["consumption"] * df_display["current_factor"]
        df_display["Adj_Stand_kWh"] = df_display["consumption"] * new_factor_val
        df_display["Verschil_kWh"] = df_display["Adj_Stand_kWh"] - df_display["Orig_Stand_kWh"]
        df_display = df_display[["period", "consumption", "current_factor", "Orig_Stand_kWh", 
                                 "Adj_Stand_kWh", "Verschil_kWh", "NettLoss", "Multiplier"]]
        df_display.rename(columns={
            "period": "Periode",
            "consumption": "Ruwe Stand (Impuls)",
            "current_factor": "Huidige Factor",
            "Orig_Stand_kWh": "Originele Stand (kWh)",
            "Adj_Stand_kWh": "Aangepaste Stand (kWh)",
            "Verschil_kWh": "Verschil (kWh)",
            "NettLoss" : "Netverlies (%)",
            "Multiplier": "Multiplier"
        }, inplace=True)

        for col in ["Originele Stand (kWh)", "Aangepaste Stand (kWh)", "Verschil (kWh)"]:
            df_display[col] = df_display[col].round(4)
        df_display["Huidige Factor"] = df_display["Huidige Factor"].round(4)
        df_display["Netverlies (%)"] = (df_display["Netverlies (%)"] * 100).round(2)

        self.update_progress(70, "Data verwerkt, tabel genereren...")
        try:
            from qgrid import QgridWidget
            qgrid_widget = QgridWidget(df=df_display, show_toolbar=True)
            display_widget = qgrid_widget
        except ImportError:
            html_table = df_display.to_html(classes=['table', 'table-striped', 'table-hover', 'table-sm'], escape=False, index=False)
            display_widget = widgets.HTML(f"""
            <div style="max-height: 600px; overflow-y: auto; width: 100%;">
            <p><b>Vergelijking van standen (vanaf {eff_date.strftime('%Y-%m-%d %H:%M')}) voor RegisterID={reg_id} met nieuwe factor {new_factor_val:.4f}</b></p>
            {html_table}
            </div>
            """)

        self.view_container.children = [display_widget]
        self.update_progress(100, "Tabel gegenereerd.")
        with self.output_widget:
            clear_output()
        self.finish_progress()

db_manager     = DatabaseManager()
business_logic = BusinessLogic(db_manager)
ui_manager     = UIManager(db_manager, business_logic)

Output()

2025-05-16 10:28:41,212 - INFO - SQL-engine aangemaakt voor inn-vee-sql12/EDS2
2025-05-16 10:28:41,222 - INFO - SQL-engine aangemaakt voor inn-vee-sql12/EDS2
2025-05-16 10:28:41,223 - INFO - SQL-engine succesvol aangemaakt.
